# Employment History

In this project we are provided a PDF file as the source (`employee_history.pdf`). This PDF file contains Tabular data, that we need to read and map to tables accordingly. This PDF has been generated using a custom dataset generator, populated with mock data.

## Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PyPDF2 as pdf

In [3]:
%matplotlib inline

## Data Collection

From looking at the pdf file, we can observe that, we have one employee per page, thus we can say that `num_employees = num_pages`.

In every page, we have two sections:
- Basic Info [Employee Name, Date Of Birth, Organization, Location, Department, Designation]
- Posting History [Designation, Department, Organization, Location, From Date, To Date, Pay grade]

For better explainability we will separately consolidate for both the sections. For each section we will perform the following steps:
- Iterate through all pages
- For each page, Search for the section title (eg. Posting History)
- Find the keywords, and populate the corresponding values as a row in the respective dataframe

After consolidation we will have two dataframes, that have a relationship through EMP_ID (employee ID):
- basic_info
- posting_history

In [10]:
file_name = 'employee_history.pdf'

pdf_file = open(file_name, 'rb')
reader = pdf.PdfFileReader(pdf_file)

num_pages = reader.numPages
num_pages

300

In [15]:
# Sample
page = reader.getPage(0)
print(page.extract_text())

Employee ID: 1
Employee Name:
Date Of Birth:
Organization:
Location:
Department
Designation:
Posting History
Designation
Department
Organization
Location
From Date
To Date
Pay Grade
Mr. Ecdwsg Jzxuatfx
1986-04-30
Cdxnpcqzb
Bspgkq, VP
Security
Security Engineer II
Security Engineer I
Security
Gzqsvu
Jzke, ER
2004-04-25
2016-10-05
1400000
Security Engineer I
Security
Jknleampe
Ivhese, JD
2016-10-06
2020-03-22
1700000
Security Engineer I
Security
Ofvyterxnkm
Yelldcaibr, BS
2020-03-23
2021-10-22
1200000
Senior Security Engineer
Security
Swgr
Qfvgfppf, JR
2021-10-23
2021-12-14
700000
Security Engineer II
Security
Cdxnpcqzb
Bspgkq, VP
2021-12-15
2022-01-06
2000000



Here when we compare the result of the reader to the pdf, we observe the following pattern:

| Line No. | Line | Type |
|----------|------|------|
|1| Employee ID | Basic Info Header |
|2| Employee Name | Header
|3| Date Of Birth | Header
|4| Organization | Header
|5| Location | Header
|6| Department | Header
|7| Designation | Header
|8| Posting History | Posting History Header
|9| Designation | Header
|10| Department | Header
|11| Organization | Header
|12| Location | Header
|13| From Date | Header
|14| To Date | Header
|15| Pay Grade | Header
|16| Mr. Ecdwsg Jzxuatfx | Basic Info / Value
|17| 1986-04-30  | Basic Info / Value
|18| Cdxnpcqzb  | Basic Info / Value
|19| Bspgkq, VP  | Basic Info / Value
|20| Security  | Basic Info / Value
|21| Security Engineer II  | Basic Info / Value
|22| Security Engineer I  | Posting History / Value
|23| Security | Posting History / Value
|24| Gzqsvu | Posting History / Value
|25| Jzke, ER | Posting History / Value
|26| 2004-04-25 | Posting History / Value
|27| 2016-10-05 | Posting History / Value
|28| 1400000 | Posting History / Value
|...|...|...|

### Basic Info

In [83]:
def get_basic_info(lines):
    page_df = pd.DataFrame()
    lines = lines.split('\n')
    emp_id = lines[0].split(': ')[1]
    ptr = 1
    col_headers = []
    while lines[ptr] != 'Posting History':
        col_headers.append(lines[ptr].strip(':'))
        ptr+=1

    ptr += 8
    row = {'ID': emp_id}
    for col in col_headers:
        row[col] = lines[ptr]
        ptr+=1

    page_df = page_df.append(row, ignore_index = True)
    
    return page_df

### Posting History

In [111]:
def get_posting_info(lines):
    page_df = pd.DataFrame()
    lines = lines.split('\n')
    emp_id = lines[0].split(': ')[1]
    ptr = 8
    col_headers = []
    while ptr < 15:
        col_headers.append(lines[ptr])
        ptr+=1

    ptr+=6
    while ptr < len(lines):
        if lines[ptr] != '':
            row = {'ID': emp_id}
            for col in col_headers:
                row[col] = lines[ptr]
                ptr+=1
        else: break
        page_df = page_df.append(row, ignore_index = True)

    return page_df
            

In [119]:
basic_info = pd.DataFrame()
posting_info = pd.DataFrame()

for i in range(num_pages):
    page = reader.getPage(i)
    lines = page.extract_text()
    basic_info = basic_info.append(get_basic_info(lines), ignore_index = True)
    posting_info = posting_info.append(get_posting_info(lines), ignore_index = True)

#basic_info.reset_index(drop=True)
#posting_info.reset_index(drop=True)

In [120]:
basic_info.head()

,ID,Employee Name,Date Of Birth,Organization,Location,Department,Designation
0,1,Mr. Ecdwsg Jzxuatfx,1986-04-30,Cdxnpcqzb,"Bspgkq, VP",Security,Security Engineer II
1,2,Mrs. Urrpzlb Auuqf,1988-07-08,Cdxnpcqzb,"Ticcgpcwxox, TA",Customer Support,Senior Support Engineer
2,3,Ms. Qfahmr Ggpwdwd,1976-10-29,Cdxnpcqzb,"Rvryjrjji, AL",Accounts,Account Manager
3,4,Mrs. Eazzqcog Kmzo,1982-01-05,Cdxnpcqzb,"Fvwbi, TS",Customer Support,Junior Support Engineer
4,5,Mrs. Noxv Rqpycrge,1995-01-01,Cdxnpcqzb,"Swskebchlvs, QE",Customer Support,Junior Support Engineer


In [121]:
posting_info.head(15)

,ID,Designation,Department,Organization,Location,From Date,To Date,Pay Grade
0,1,Security Engineer I,Security,Gzqsvu,"Jzke, ER",2004-04-25,2016-10-05,1400000
1,1,Security Engineer I,Security,Jknleampe,"Ivhese, JD",2016-10-06,2020-03-22,1700000
2,1,Security Engineer I,Security,Ofvyterxnkm,"Yelldcaibr, BS",2020-03-23,2021-10-22,1200000
3,1,Senior Security Engineer,Security,Swgr,"Qfvgfppf, JR",2021-10-23,2021-12-14,700000
4,1,Security Engineer II,Security,Cdxnpcqzb,"Bspgkq, VP",2021-12-15,2022-01-06,2000000
5,2,Junior Support Engineer,Customer Support,Mohixzup,"Rvryjrjji, AL",2006-07-04,2008-04-20,2000000
6,2,Customer Support Manager,Customer Support,Oqcugctmdyxo,"Whlay, PN",2008-04-21,2019-10-31,1700000
7,2,Customer Support Manager,Customer Support,Zihrqct,"Lmjdtngzzc, DZ",2019-11-01,2021-11-01,1500000
8,2,Junior Support Engineer,Customer Support,Ilgnytduxagz,"Alennxymgmxb, TA",2021-11-02,2021-11-26,800000
9,2,Junior Support Engineer,Customer Support,Zrcilrlvcz,"Nfhlqopggrmu, LV",2021-11-27,2021-12-30,2000000


## Data Preprocessing and Feature Engineering

We have organised the data into 2 dataframes: `basic_info` and `posting_history`

By doing some transformations, we can simplify our data further and highlight certain specific aspects of it. We will do the following procedures:

- Check dtypes: Change (str -> int), (str -> datetime) whereever applicable
- Gender Labels (M/F): Based on the title available as part of the `Employee Name`. {'Mr' -> M, 'Mrs/Ms' -> F}
- City and State Labels (Abcd, XY): To be extracted from `Location`.
- Year: Extract from `Date Of Birth` (Birth Year), `From Date` (From Year) and `To Date` (To Year)

### Data type validation

In [122]:
basic_info.dtypes

ID               object
Employee Name    object
Date Of Birth    object
Organization     object
Location         object
Department       object
Designation      object
dtype: object

In [125]:
basic_info['ID'] = basic_info['ID'].astype(int)
basic_info['Date Of Birth'] = pd.to_datetime(basic_info['Date Of Birth'])
basic_info.dtypes

ID                        int32
Employee Name            object
Date Of Birth    datetime64[ns]
Organization             object
Location                 object
Department               object
Designation              object
dtype: object

In [127]:
posting_info.dtypes

ID              object
Designation     object
Department      object
Organization    object
Location        object
From Date       object
To Date         object
Pay Grade       object
dtype: object

In [128]:
posting_info['ID'] = posting_info['ID'].astype(int)
posting_info['Pay Grade'] = posting_info['Pay Grade'].astype(int)
posting_info['From Date'] = pd.to_datetime(posting_info['From Date'])
posting_info['To Date'] = pd.to_datetime(posting_info['To Date'])
posting_info.dtypes

ID                       int32
Designation             object
Department              object
Organization            object
Location                object
From Date       datetime64[ns]
To Date         datetime64[ns]
Pay Grade                int32
dtype: object

### Gender Labels

In [132]:
basic_info['Gender'] = basic_info['Employee Name'].apply(lambda x: 'M' if x.split('.')[0] == 'Mr' else 'F')
basic_info

,ID,Employee Name,Date Of Birth,Organization,Location,Department,Designation,Gender
0,1,Mr. Ecdwsg Jzxuatfx,1986-04-30,Cdxnpcqzb,"Bspgkq, VP",Security,Security Engineer II,M
1,2,Mrs. Urrpzlb Auuqf,1988-07-08,Cdxnpcqzb,"Ticcgpcwxox, TA",Customer Support,Senior Support Engineer,F
2,3,Ms. Qfahmr Ggpwdwd,1976-10-29,Cdxnpcqzb,"Rvryjrjji, AL",Accounts,Account Manager,F
3,4,Mrs. Eazzqcog Kmzo,1982-01-05,Cdxnpcqzb,"Fvwbi, TS",Customer Support,Junior Support Engineer,F
4,5,Mrs. Noxv Rqpycrge,1995-01-01,Cdxnpcqzb,"Swskebchlvs, QE",Customer Support,Junior Support Engineer,F
...,...,...,...,...,...,...,...,...
295,296,Mrs. Yxasuaam Lfgvanbn,1993-08-27,Cdxnpcqzb,"Xojnox, VW",IT & Operations,IT Consultant,F
296,297,Mrs. Epxesz Dwmtewnt,1970-03-07,Cdxnpcqzb,"Pocynryfoiv, OR",IT & Operations,IT Consultant,F
297,298,Mr. Qykivm Dmnvpyr,1992-09-05,Cdxnpcqzb,"Whlay, PN",Human Resources,HR,M
298,299,Mr. Kugvnc Vjngvcj,1998-09-08,Cdxnpcqzb,"Wftnugiqu, EZ",Security,Security Engineer I,M


### City/State Labels

In [142]:
# Alternative Approach:
# basic_info['City'] = basic_info['Location'].str.split(', ')[0]
# basic_info['State'] = basic_info['Location'].str.split(', ')[1]

In [133]:
basic_info['City'] = basic_info['Location'].apply(lambda x: x.split(',')[0])
basic_info.head()

,ID,Employee Name,Date Of Birth,Organization,Location,Department,Designation,Gender,City
0,1,Mr. Ecdwsg Jzxuatfx,1986-04-30,Cdxnpcqzb,"Bspgkq, VP",Security,Security Engineer II,M,Bspgkq
1,2,Mrs. Urrpzlb Auuqf,1988-07-08,Cdxnpcqzb,"Ticcgpcwxox, TA",Customer Support,Senior Support Engineer,F,Ticcgpcwxox
2,3,Ms. Qfahmr Ggpwdwd,1976-10-29,Cdxnpcqzb,"Rvryjrjji, AL",Accounts,Account Manager,F,Rvryjrjji
3,4,Mrs. Eazzqcog Kmzo,1982-01-05,Cdxnpcqzb,"Fvwbi, TS",Customer Support,Junior Support Engineer,F,Fvwbi
4,5,Mrs. Noxv Rqpycrge,1995-01-01,Cdxnpcqzb,"Swskebchlvs, QE",Customer Support,Junior Support Engineer,F,Swskebchlvs


In [134]:
basic_info['State'] = basic_info['Location'].apply(lambda x: x.split(', ')[1])
basic_info.head()

,ID,Employee Name,Date Of Birth,Organization,Location,Department,Designation,Gender,City,State
0,1,Mr. Ecdwsg Jzxuatfx,1986-04-30,Cdxnpcqzb,"Bspgkq, VP",Security,Security Engineer II,M,Bspgkq,VP
1,2,Mrs. Urrpzlb Auuqf,1988-07-08,Cdxnpcqzb,"Ticcgpcwxox, TA",Customer Support,Senior Support Engineer,F,Ticcgpcwxox,TA
2,3,Ms. Qfahmr Ggpwdwd,1976-10-29,Cdxnpcqzb,"Rvryjrjji, AL",Accounts,Account Manager,F,Rvryjrjji,AL
3,4,Mrs. Eazzqcog Kmzo,1982-01-05,Cdxnpcqzb,"Fvwbi, TS",Customer Support,Junior Support Engineer,F,Fvwbi,TS
4,5,Mrs. Noxv Rqpycrge,1995-01-01,Cdxnpcqzb,"Swskebchlvs, QE",Customer Support,Junior Support Engineer,F,Swskebchlvs,QE


In [135]:
posting_info['City'] = posting_info['Location'].apply(lambda x: x.split(', ')[0])
posting_info['State'] = posting_info['Location'].apply(lambda x: x.split(', ')[1])
posting_info.head()

,ID,Designation,Department,Organization,Location,From Date,To Date,Pay Grade,City,State
0,1,Security Engineer I,Security,Gzqsvu,"Jzke, ER",2004-04-25,2016-10-05,1400000,Jzke,ER
1,1,Security Engineer I,Security,Jknleampe,"Ivhese, JD",2016-10-06,2020-03-22,1700000,Ivhese,JD
2,1,Security Engineer I,Security,Ofvyterxnkm,"Yelldcaibr, BS",2020-03-23,2021-10-22,1200000,Yelldcaibr,BS
3,1,Senior Security Engineer,Security,Swgr,"Qfvgfppf, JR",2021-10-23,2021-12-14,700000,Qfvgfppf,JR
4,1,Security Engineer II,Security,Cdxnpcqzb,"Bspgkq, VP",2021-12-15,2022-01-06,2000000,Bspgkq,VP


### Year Extraction

In [136]:
basic_info['Birth Year'] = basic_info['Date Of Birth'].apply(lambda x: x.year)
basic_info.head()

,ID,Employee Name,Date Of Birth,Organization,Location,Department,Designation,Gender,City,State,Birth Year
0,1,Mr. Ecdwsg Jzxuatfx,1986-04-30,Cdxnpcqzb,"Bspgkq, VP",Security,Security Engineer II,M,Bspgkq,VP,1986
1,2,Mrs. Urrpzlb Auuqf,1988-07-08,Cdxnpcqzb,"Ticcgpcwxox, TA",Customer Support,Senior Support Engineer,F,Ticcgpcwxox,TA,1988
2,3,Ms. Qfahmr Ggpwdwd,1976-10-29,Cdxnpcqzb,"Rvryjrjji, AL",Accounts,Account Manager,F,Rvryjrjji,AL,1976
3,4,Mrs. Eazzqcog Kmzo,1982-01-05,Cdxnpcqzb,"Fvwbi, TS",Customer Support,Junior Support Engineer,F,Fvwbi,TS,1982
4,5,Mrs. Noxv Rqpycrge,1995-01-01,Cdxnpcqzb,"Swskebchlvs, QE",Customer Support,Junior Support Engineer,F,Swskebchlvs,QE,1995


In [141]:
posting_info['From Year'] = posting_info['From Date'].dt.year
posting_info['To Year'] = posting_info['To Date'].dt.year
posting_info.head()

,ID,Designation,Department,Organization,Location,From Date,To Date,Pay Grade,City,State,From Year,To Year
0,1,Security Engineer I,Security,Gzqsvu,"Jzke, ER",2004-04-25,2016-10-05,1400000,Jzke,ER,2004,2016
1,1,Security Engineer I,Security,Jknleampe,"Ivhese, JD",2016-10-06,2020-03-22,1700000,Ivhese,JD,2016,2020
2,1,Security Engineer I,Security,Ofvyterxnkm,"Yelldcaibr, BS",2020-03-23,2021-10-22,1200000,Yelldcaibr,BS,2020,2021
3,1,Senior Security Engineer,Security,Swgr,"Qfvgfppf, JR",2021-10-23,2021-12-14,700000,Qfvgfppf,JR,2021,2021
4,1,Security Engineer II,Security,Cdxnpcqzb,"Bspgkq, VP",2021-12-15,2022-01-06,2000000,Bspgkq,VP,2021,2022
